# Cell 0: Environment & Backend

In [ ]:
import os, sys, importlib, pathlib
from datetime import datetime

USE_WIDGETS   = int(os.getenv("USE_WIDGETS", "0"))
ENABLE_ACTIVE = int(os.getenv("ENABLE_ACTIVE", "0"))

PROJECT_ROOT = pathlib.Path.cwd()
DATA_RAW     = (PROJECT_ROOT / "data" / "raw").resolve()
DATA_PROC    = (PROJECT_ROOT / "data" / "processed").resolve()
RESULTS_ROOT = (PROJECT_ROOT / "results").resolve()
for d in (DATA_RAW, DATA_PROC, RESULTS_ROOT):
    d.mkdir(parents=True, exist_ok=True)

def _enable_inline():
    get_ipython().run_line_magic("matplotlib", "inline")
def _enable_widget():
    get_ipython().run_line_magic("matplotlib", "widget")

widgets_enabled = False
if USE_WIDGETS:
    try:
        import ipywidgets, ipympl 
        _enable_widget()
        widgets_enabled = True
    except Exception as e:
        print(f"[warn] USE_WIDGETS=1 but ipywidgets/ipympl not available "
              f"({e.__class__.__name__}). Falling back to inline.")
        _enable_inline()
else:
    _enable_inline()

import matplotlib

def _ver(modname):
    try:
        m = importlib.import_module(modname)
        return getattr(m, "__version__", "unknown")
    except Exception as e:
        return f"missing ({e.__class__.__name__})"

print("=== Environment ===")
print("Time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Python:", sys.executable)
print("Matplotlib backend:", matplotlib.get_backend())
print("Widgets enabled:", widgets_enabled)
print("numpy:", _ver("numpy"))
print("opencv-python (cv2):", _ver("cv2"))
print("scikit-image:", _ver("skimage"))
print("scikit-learn:", _ver("sklearn"))
print("ipywidgets:", _ver("ipywidgets"))
print("ipympl:", _ver("ipympl"))
print("\nDATA_RAW:", str(DATA_RAW))
print("DATA_PROC:", str(DATA_PROC))
print("RESULTS_ROOT:", str(RESULTS_ROOT))


# Cell 1: Paths & Imports

In [ ]:
import os, cv2, numpy as np, matplotlib.pyplot as plt
from IPython.display import display
from pathlib import Path

PROJECT_ROOT = Path(PROJECT_ROOT)
DATA_RAW     = Path(DATA_RAW)
DATA_PROC    = Path(DATA_PROC)
RESULTS_ROOT = Path(RESULTS_ROOT)

for d in [DATA_RAW, DATA_PROC, RESULTS_ROOT]:
    d.mkdir(parents=True, exist_ok=True)

print("RAW:", str(DATA_RAW))
print("PROC:", str(DATA_PROC))
print("RESULTS:", str(RESULTS_ROOT))

if not DATA_RAW.is_dir():
    raise FileNotFoundError(f"RAW directory not found: {DATA_RAW}")

VALID_EXT = (".jpg", ".jpeg", ".png", ".tif", ".tiff", ".bmp")


# Cell 2: Load & Preview Raw Images

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt

images = []  
print(f"Using DATA_RAW: {DATA_RAW}")
fnames = sorted([p.name for p in DATA_RAW.iterdir() if p.suffix.lower() in VALID_EXT])
print(f"Found {len(fnames)} image(s)")

for fname in fnames:
    path = DATA_RAW / fname
    img_bgr = cv2.imread(str(path), cv2.IMREAD_COLOR)  
    if img_bgr is None:
        print(f"[warn] failed to read {path}, skipping.")
        continue
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    images.append((fname, img_rgb))

print(f"Loaded {len(images)} image(s).")

n = len(images)
if n:
    show_n = min(n, 9)
    cols = min(3, show_n)
    rows = int(np.ceil(show_n/cols))
    fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
    axes = np.atleast_1d(axes).ravel()
    for ax, (fname, img) in zip(axes, images[:show_n]):
        ax.imshow(img); ax.set_title(fname); ax.axis("off")
    for ax in axes[show_n:]:
        ax.axis("off")
    plt.tight_layout(); plt.show()
else:
    print("[note] No images loaded — add files to data/raw and re-run.")


# Cell 3: Preprocess (Grayscale, Resize, Equalize)

In [ ]:
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt

assert 'images' in globals() and len(images) > 0, "Run Cell 2 first to populate `images`."

PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
TARGET_SIZE = (512, 512)

preprocessed = []

for fname, img_rgb in images:
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    gray_resized = cv2.resize(gray, TARGET_SIZE, interpolation=cv2.INTER_AREA)
    gray_blur = cv2.GaussianBlur(gray_resized, (3, 3), 0)
    gray_eq = cv2.equalizeHist(gray_blur)

    preprocessed.append((fname, gray_eq))
    cv2.imwrite(str(PROCESSED_DIR / f"{Path(fname).stem}_gray_eq.png"), gray_eq)

print(f"Preprocessed {len(preprocessed)} image(s) → {PROCESSED_DIR}")

n = len(preprocessed)
cols = min(3, max(1, int(math.ceil(n**0.5))))
rows = int(math.ceil(n / cols))
fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 4*rows))
axes = np.atleast_1d(axes).ravel()

for ax, (fname, gray_eq) in zip(axes, preprocessed):
    ax.imshow(gray_eq, cmap="gray")
    ax.set_title(f"{fname} • preprocessed")
    ax.axis("off")

for ax in axes[len(preprocessed):]:
    ax.axis("off")

plt.tight_layout()
plt.show()


# Cell 3.5: Utilities & Feature Cache

In [ ]:
import numpy as np, cv2

try:
    from skimage.filters import rank
    from skimage.morphology import disk
    _HAS_SKIMAGE = True
except Exception:
    _HAS_SKIMAGE = False

def ensure_uint8(img):
    if img.dtype != np.uint8:
        return np.clip(img, 0, 255).astype(np.uint8)
    return img

def feature_stack(img_rgb, gray):
    """
    Build HxWxF feature cube aligned to the preprocessed gray size (H,W).
    Features: [gray, r_n, g_n, b_n, VI_proxy, laplacian, grad_mag, entropy_or_var]
    """
    gray = ensure_uint8(gray)
    H, W = gray.shape[:2]

    if img_rgb.ndim == 2:
        img_rgb = np.stack([img_rgb]*3, axis=-1)
    elif img_rgb.shape[2] == 4:
        img_rgb = img_rgb[..., :3]
    elif img_rgb.shape[2] == 1:
        img_rgb = np.concatenate([img_rgb]*3, axis=-1)
    else:
        img_rgb = img_rgb[..., :3]

    if img_rgb.shape[:2] != (H, W):
        img_rgb = cv2.resize(img_rgb, (W, H), interpolation=cv2.INTER_AREA)

    rgb = img_rgb.astype(np.float32)
    s = np.maximum(rgb.sum(axis=2), 1e-6)
    r_n = rgb[..., 0] / s
    g_n = rgb[..., 1] / s
    b_n = rgb[..., 2] / s

    vi = (rgb[..., 1] - rgb[..., 0]) / (rgb[..., 1] + rgb[..., 0] + 1e-6)

    lap  = cv2.Laplacian(gray, cv2.CV_32F, ksize=3)
    sobx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    soby = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    grad = np.hypot(sobx, soby)

    if _HAS_SKIMAGE:
        ent = rank.entropy(ensure_uint8(gray), disk(5)).astype(np.float32)
    else:
        ent = cv2.blur((cv2.Laplacian(gray, cv2.CV_32F, ksize=3) ** 2), (5, 5))

    F = np.dstack([
        gray.astype(np.float32),
        r_n.astype(np.float32), g_n.astype(np.float32), b_n.astype(np.float32),
        vi.astype(np.float32), lap, grad, ent
    ])
    return F

def clean_mask(mask, min_area=200, k=3):
    m = (mask > 0).astype(np.uint8) * 255
    kernel = np.ones((k, k), np.uint8)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN,  kernel, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kernel, iterations=1)
    num, lab, stats, _ = cv2.connectedComponentsWithStats((m > 0).astype(np.uint8), connectivity=8)
    out = np.zeros_like(m)
    for i in range(1, num):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            out[lab == i] = 255
    out = cv2.medianBlur(out, 3)
    return out

def get_gray_eq(fname):
    for f, g in preprocessed:
        if f == fname:
            return g
    raise KeyError(f"No preprocessed gray for {fname}")

FEATURES = {}
if not images:
    print("No images found; feature cache skipped.")
else:
    for fname, img_rgb in images:
        F = feature_stack(img_rgb, get_gray_eq(fname))
        FEATURES[fname] = (F,) + F.shape
    print(f"Cached features for {len(FEATURES)} images.")


# Cell 4: Baseline Forest Classification – Thresholding (Otsu)

In [ ]:
RESULTS_THRESH = os.path.join(RESULTS_ROOT, "thresholding")
os.makedirs(RESULTS_THRESH, exist_ok=True)

otsu_results = []
for fname, gray_eq in preprocessed:
    _, otsu = cv2.threshold(gray_eq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if (otsu > 0).sum() / otsu.size < 0.3:
        otsu = 255 - otsu
    otsu_results.append((fname, otsu))

    plt.figure(figsize=(5,5))
    plt.title(f"Otsu: {fname}")
    plt.imshow(otsu, cmap="gray"); plt.axis("off")
    plt.show(); plt.close()

    cv2.imwrite(os.path.join(RESULTS_THRESH, f"{os.path.splitext(fname)[0]}_otsu.png"), otsu)

print(f"Otsu masks saved to: {RESULTS_THRESH}")


# Cell 5: Baseline Canny + Morphology

In [ ]:
RESULTS_EDGES = os.path.join(RESULTS_ROOT, "edges")
os.makedirs(RESULTS_EDGES, exist_ok=True)

edges_results = []
for fname, gray_eq in preprocessed:
    edges = cv2.Canny(gray_eq, 100, 200)
    kernel = np.ones((3,3), np.uint8)
    mask = cv2.dilate(edges, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    edges_results.append((fname, mask))

    fig, ax = plt.subplots(1,2, figsize=(10,4))
    ax[0].set_title(f"Canny: {fname}"); ax[0].imshow(edges, cmap="gray"); ax[0].axis("off")
    ax[1].set_title("Edge Region Mask"); ax[1].imshow(mask,  cmap="gray"); ax[1].axis("off")
    plt.show(); plt.close()

    cv2.imwrite(os.path.join(RESULTS_EDGES, f"{os.path.splitext(fname)[0]}_edges_mask.png"), mask)

print(f"Edge-region masks saved to: {RESULTS_EDGES}")


# Cell 6: Baseline Forest Classification – K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

RESULTS_KMEANS = os.path.join(RESULTS_ROOT, "kmeans")
os.makedirs(RESULTS_KMEANS, exist_ok=True)

kmeans_results = []
for fname, img_rgb in images:
    gray_eq = get_gray_eq(fname)
    F = feature_stack(img_rgb, gray_eq)
    H, W, C = F.shape
    X = F.reshape(-1, C)
    Xs = StandardScaler().fit_transform(X)

    km = KMeans(n_clusters=2, n_init=10, random_state=42).fit(Xs)
    lab = km.labels_.reshape(H, W)

    r_n = F[...,1]; vi = F[...,4]
    m_vi0 = vi[lab==0].mean() if (lab==0).any() else -1e9
    m_vi1 = vi[lab==1].mean() if (lab==1).any() else -1e9
    forest_label = 0 if m_vi0 > m_vi1 else 1

    mask = (lab == forest_label).astype(np.uint8)*255
    mask = clean_mask(mask)

    kmeans_results.append((fname, mask))
    plt.figure(figsize=(5,5))
    plt.title(f"K-Means (forest): {fname}")
    plt.imshow(mask, cmap="gray"); plt.axis("off")
    plt.show(); plt.close()

    cv2.imwrite(os.path.join(RESULTS_KMEANS, f"{os.path.splitext(fname)[0]}_kmeans_forest.png"), mask)

print(f"K-Means masks saved to: {RESULTS_KMEANS}")


# Cell 7: Side-By-Side Comparison

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt

otsu_map = {f: m for f, m in otsu_results}
edge_map = {f: m for f, m in edges_results}
km_map   = {f: m for f, m in kmeans_results}

shown = 0
for fname, img_rgb in images:
    if (fname not in otsu_map) or (fname not in edge_map) or (fname not in km_map):
        print(f"[skip] {fname}: missing a baseline result.")
        continue

    fig, axs = plt.subplots(1, 4, figsize=(16, 5), constrained_layout=True)
    axs[0].imshow(img_rgb);            axs[0].set_title(f"Original: {fname}"); axs[0].axis("off")
    axs[1].imshow(otsu_map[fname],  cmap="gray"); axs[1].set_title("Otsu");         axs[1].axis("off")
    axs[2].imshow(edge_map[fname],  cmap="gray"); axs[2].set_title("Canny+Morph");  axs[2].axis("off")
    axs[3].imshow(km_map[fname],    cmap="gray"); axs[3].set_title("K-Means");      axs[3].axis("off")

    display(fig)
    plt.close(fig)
    shown += 1

print(f"Displayed {shown} comparison figure(s).")


# Cell 8: Quantitative Comparison

In [ ]:
def forest_fraction(mask):
    return (mask > 0).sum() / mask.size

print("Forest pixel fraction (per method):\n")
for fname, _ in images:
    if (fname not in otsu_map) or (fname not in edge_map) or (fname not in km_map):
        continue
    print(fname)
    print("  Otsu   :", round(forest_fraction(otsu_map[fname]), 3))
    print("  Canny  :", round(forest_fraction(edge_map[fname]), 3))
    print("  K-Means:", round(forest_fraction(km_map[fname]), 3))
    print()


# Cell 9: Auto Generated Clicks

In [ ]:
import json

N_POS, N_NEG = 60, 60
SEED  = 42
rng   = np.random.default_rng(SEED)

def to_bin(m): return (m > 0).astype(np.uint8)
def interior(m, k=3, iters=1):
    m = m.astype(np.uint8)
    kern = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k,k))
    return cv2.erode(m, kern, iterations=iters)
def open_thin(m, k=3, iters=1):
    m = m.astype(np.uint8)
    kern = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k,k))
    return cv2.morphologyEx(m, cv2.MORPH_OPEN, kern, iterations=iters)
def sample_coords(mask_bin, n, rng):
    ys, xs = np.where(mask_bin > 0)
    if ys.size == 0 or n <= 0: return []
    n = min(n, ys.size)
    idx = rng.choice(ys.size, size=n, replace=False)
    return list(zip(xs[idx], ys[idx]))
def crop_border_mask(h, w, pad_px):
    keep = np.zeros((h, w), np.uint8)
    keep[pad_px:h-pad_px, pad_px:w-pad_px] = 1
    return keep

otsu_map = {f: m for f, m in otsu_results}
km_map   = {f: m for f, m in kmeans_results}

CLICK_SAMPLES = []  
total_pos = total_neg = 0

for fname, _ in images:
    if fname not in otsu_map or fname not in km_map:
        print(f"[warn] {fname}: missing masks; skipping.")
        continue

    A = to_bin(km_map[fname])
    B = to_bin(otsu_map[fname])

    H = min(A.shape[0], B.shape[0]); W = min(A.shape[1], B.shape[1])
    A, B = A[:H, :W], B[:H, :W]

    pad = max(10, int(0.03 * min(H, W)))
    keep = crop_border_mask(H, W, pad)
    A &= keep; B &= keep

    I  = (A & B).astype(np.uint8)       
    K  = (A & (~B)).astype(np.uint8)    
    O  = (B & (~A)).astype(np.uint8)    
    BG = (~(A | B)).astype(np.uint8)    

    I  = open_thin(I, 3, 1)
    K  = open_thin(K, 3, 1)
    O  = open_thin(O, 3, 1)
    BG = open_thin(BG, 3, 1)

    nI, nK = N_POS // 2, N_POS - (N_POS // 2)
    nBG, nO = N_NEG // 2, N_NEG - (N_NEG // 2)

    pos = sample_coords(interior(I, 3, 1), nI,  rng) + sample_coords(interior(K, 3, 1), nK, rng)
    neg = sample_coords(interior(BG,3, 1), nBG, rng) + sample_coords(interior(O, 3, 1), nO, rng)

    if len(pos) < N_POS:
        pos_all = interior((A | B).astype(np.uint8), 3, 1)
        pos += sample_coords(pos_all, N_POS - len(pos), rng)
    if len(neg) < N_NEG:
        neg_all = interior((BG | O).astype(np.uint8), 3, 1)
        neg += sample_coords(neg_all, N_NEG - len(neg), rng)

    if len(pos) == 0 or len(neg) == 0:
        print(f"[warn] {fname}: not enough candidates; skipping.")
        continue

    for (x, y) in pos[:N_POS]:
        CLICK_SAMPLES.append({"fname": fname, "x": int(x), "y": int(y), "label": 1, "source": "auto"})
    for (x, y) in neg[:N_NEG]:
        CLICK_SAMPLES.append({"fname": fname, "x": int(x), "y": int(y), "label": 0, "source": "auto"})

    total_pos += min(len(pos), N_POS)
    total_neg += min(len(neg), N_NEG)
    print(f"{fname}: +{min(len(pos), N_POS)} pos, +{min(len(neg), N_NEG)} neg")

print(f"\nAuto-generated clicks total: {total_pos} pos, {total_neg} neg (~{N_POS}+{N_NEG} per image)")

(PROJECT_ROOT / "data").mkdir(parents=True, exist_ok=True)
CLICKS_PATH = PROJECT_ROOT / "data" / "training_clicks.json"
with open(str(CLICKS_PATH), "w") as f:
    json.dump(CLICK_SAMPLES, f)
print("Saved to:", str(CLICKS_PATH))
print("CLICK_SAMPLES in RAM:", len(CLICK_SAMPLES))


# Cell 10: Build Dataset

In [ ]:
import json
from collections import Counter

WEIGHT_BY_SOURCE = { "auto": 1.0, "active": 3.0 }
CLICKS_PATH = PROJECT_ROOT / "data" / "training_clicks.json"

def load_clicks():
    if not CLICKS_PATH.exists():
        return []
    with open(str(CLICKS_PATH)) as f:
        arr = json.load(f)
    for c in arr:
        if "source" not in c:
            c["source"] = "auto"
    return arr

def save_clicks(clicks):
    CLICKS_PATH.parent.mkdir(parents=True, exist_ok=True)
    with open(str(CLICKS_PATH), "w") as f:
        json.dump(clicks, f)
    print(f"[saved] {len(clicks)} clicks -> {str(CLICKS_PATH)}")

def dedupe_clicks(clicks):
    seen = set(); out = []
    for c in clicks:
        key = (c["fname"], int(c["x"]), int(c["y"]))
        if key in seen:
            continue
        seen.add(key); out.append(c)
    return out

CLICK_SAMPLES = dedupe_clicks((globals().get("CLICK_SAMPLES") or []) + load_clicks())
save_clicks(CLICK_SAMPLES)
print("Clicks by source:", {s: sum(1 for c in CLICK_SAMPLES if c["source"]==s) 
                            for s in set([c["source"] for c in CLICK_SAMPLES])})

rgb_map = {f: img for f, img in images}

def build_training_xy_weighted(clicks):
    X_list, y_list, g_list, w_list = [], [], [], []
    for c in clicks:
        fname, x, y, lbl = c["fname"], int(c["x"]), int(c["y"]), int(c["label"])
        src  = c.get("source", "auto")
        if 'FEATURES' in globals() and fname in FEATURES:
            F, H, W, C = FEATURES[fname]
        else:
            img_rgb = rgb_map[fname]
            gray_eq = get_gray_eq(fname)
            F = feature_stack(img_rgb, gray_eq); H, W, C = F.shape
        if 0 <= x < W and 0 <= y < H:
            X_list.append(F[y, x, :]); y_list.append(lbl); g_list.append(fname)
            w_list.append(float(WEIGHT_BY_SOURCE.get(src, 1.0)))
    X = np.array(X_list, dtype=np.float32)
    y = np.array(y_list, dtype=np.uint8)
    groups = np.array(g_list)
    weights = np.array(w_list, dtype=np.float32)
    return X, y, groups, weights

X, y, groups, weights = build_training_xy_weighted(CLICK_SAMPLES)
print("X:", X.shape, "| y:", y.shape, "| groups:", len(groups), "| weights range:", (weights.min(), weights.max()))
print("Label counts:", Counter(y.tolist()))
assert len(np.unique(y)) == 2, "Need both classes in clicks (forest and non-forest)."


# Cell 11: Train RF & Apply

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_curve, f1_score
import numpy as np, os, cv2

if 'otsu_map' not in globals():
    otsu_map = {f: m for f, m in otsu_results}
if 'km_map' not in globals():
    km_map   = {f: m for f, m in kmeans_results}

def forest_fraction(mask_u8):
    return (mask_u8 > 0).sum() / mask_u8.size

def baseline_forest_fraction(fname, alpha=0.8):
    """
    Heuristic target fraction using Otsu (dominant) + KM (only if not extreme).
    Clamped to [0.45, 0.60].
    """
    f_otsu = forest_fraction(otsu_map[fname]) if fname in otsu_map else None
    f_km   = forest_fraction(km_map[fname])   if fname in km_map   else None
    if f_otsu is None and f_km is None:
        tgt = 0.50
    elif f_km is None or (f_km < 0.05 or f_km > 0.75):
        tgt = f_otsu if f_otsu is not None else 0.50
    else:
        tgt = alpha * f_otsu + (1.0 - alpha) * f_km
    return float(np.clip(tgt, 0.45, 0.60))

def light_clean(mask_u8):
    return clean_mask(mask_u8, min_area=150, k=3)

def frac_after_clean(p_hw, thr):
    m = ((p_hw >= thr).astype(np.uint8) * 255)
    m = light_clean(m)
    return forest_fraction(m)

def find_thr_for_target(p_hw, tgt, lo=0.10, hi=0.90, tol=0.005, max_iter=18):
    """
    Binary search a threshold so that fraction_after_clean(thr) ~ tgt.
    Returns (thr_found, frac_found).
    """
    f_lo = frac_after_clean(p_hw, lo)
    f_hi = frac_after_clean(p_hw, hi)
    if f_lo < tgt and f_hi < tgt:
        return lo, f_lo
    if f_lo > tgt and f_hi > tgt:
        return hi, f_hi
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        f_mid = frac_after_clean(p_hw, mid)
        if abs(f_mid - tgt) <= tol:
            return mid, f_mid
        if f_mid > tgt:
            lo, f_lo = mid, f_mid
        else:
            hi, f_hi = mid, f_mid
    mid = 0.5 * (lo + hi)
    return mid, frac_after_clean(p_hw, mid)

def train_rf_grouped(X, y, groups, weights, threshold_by_f1=True):
    gss = GroupShuffleSplit(n_splits=1, test_size=0.33, random_state=42)
    tr_idx, te_idx = next(gss.split(X, y, groups))
    X_tr, X_te = X[tr_idx], X[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]
    w_tr       = weights[tr_idx]

    clf = RandomForestClassifier(
        n_estimators=200,
        class_weight="balanced",
        max_features=0.5,
        n_jobs=-1,
        random_state=42
    )
    clf.fit(X_tr, y_tr, sample_weight=w_tr)
    print(classification_report(y_te, clf.predict(X_te), digits=3))

    thr = 0.5
    if threshold_by_f1 and hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X_te)[:, 1]
        prec, rec, t = precision_recall_curve(y_te, proba)
        if len(t):
            f1s = [f1_score(y_te, (proba >= tt).astype(int)) for tt in t]
            thr = float(t[int(np.argmax(f1s))])
        else:
            thr = 0.5
    thr = float(np.clip(thr, 0.25, 0.70))  
    print(f"Chosen threshold (global, clamped): {thr:.3f}")
    return clf, thr

clf, best_thr = train_rf_grouped(X, y, groups, weights)

RESULTS_RF = os.path.join(RESULTS_ROOT, "random_forest_hybrid")
os.makedirs(RESULTS_RF, exist_ok=True)
rf_results = []

print("\nPer-image thresholds and fractions (post-clean):")
print("fname, tgt, thr_img, frac_rf")

for fname, img_rgb in images:
    if 'FEATURES' in globals() and fname in FEATURES:
        F, H, W, C = FEATURES[fname]
    else:
        gray_eq = get_gray_eq(fname)
        F = feature_stack(img_rgb, gray_eq); H, W, C = F.shape

    if hasattr(clf, "predict_proba"):
        p = clf.predict_proba(F.reshape(-1, C).astype(np.float32))[:, 1].reshape(H, W)

        tgt = baseline_forest_fraction(fname)
        thr_img, _ = find_thr_for_target(p, tgt, lo=0.10, hi=0.90, tol=0.003, max_iter=20)

        pred01 = (p >= thr_img).astype(np.uint8)
        pred = (pred01 * 255).astype(np.uint8)
        pred = light_clean(pred)
    else:
        tgt, thr_img = np.nan, np.nan
        pred = (clf.predict(F.reshape(-1, C)).astype(np.uint8).reshape(H, W) * 255)
        pred = light_clean(pred)

    rf_results.append((fname, pred))

    out_path = os.path.join(RESULTS_RF, f"{os.path.splitext(fname)[0]}_rf_mask.png")
    cv2.imwrite(out_path, pred)

    thr_str = f"{thr_img:.3f}" if (isinstance(thr_img, (float, np.floating)) and not np.isnan(thr_img)) else "n/a"
    print(f"{fname}, {tgt:.3f}, {thr_str}, {forest_fraction(pred):.3f}")

print(f"\n[done] RF hybrid masks -> {RESULTS_RF}")


# Cell 12: Refine RF

In [ ]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import cv2

RESULTS_RF_REFINED = RESULTS_ROOT / "random_forest_refined"
RESULTS_RF_REFINED.mkdir(parents=True, exist_ok=True)

def fill_small_holes(m01, max_area=1500):
    inv = 1 - m01
    h, w = inv.shape
    ff = inv.copy()
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(ff, mask, (0, 0), 1)
    holes = (1 - ff).astype(np.uint8)

    num, lab, stats, _ = cv2.connectedComponentsWithStats(holes, connectivity=8)
    keep = np.zeros_like(holes, np.uint8)
    for i in range(1, num):
        if stats[i, cv2.CC_STAT_AREA] <= max_area:
            keep[lab == i] = 1
    return np.clip(m01 + keep, 0, 1)

def refine_mask(mask_u8, border_pad=10, min_area=800, close_len=7, open_len=3, hole_max=1500):
    m = (mask_u8 > 0).astype(np.uint8)
    if min(m.shape) > 2 * border_pad:
        m[:border_pad, :] = 0; m[-border_pad:, :] = 0
        m[:, :border_pad] = 0; m[:, -border_pad:] = 0

    kh = cv2.getStructuringElement(cv2.MORPH_RECT, (close_len, 1))
    kv = cv2.getStructuringElement(cv2.MORPH_RECT, (1, close_len))
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kh, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, kv, iterations=1)

    m = fill_small_holes(m, max_area=hole_max)

    kh2 = cv2.getStructuringElement(cv2.MORPH_RECT, (open_len, 1))
    kv2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, open_len))
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, kh2, iterations=1)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, kv2, iterations=1)

    m = clean_mask((m * 255).astype(np.uint8), min_area=min_area, k=3)
    return m

def forest_fraction(mask_u8):
    return (mask_u8 > 0).sum() / mask_u8.size

rf_refined_results = []
print("Refined fractions per image:")
for fname, pred in rf_results:
    refined = refine_mask(
        pred,
        border_pad=10,
        min_area=800,
        close_len=7,
        open_len=3,
        hole_max=1500
    )
    rf_refined_results.append((fname, refined))
    cv2.imwrite(str(RESULTS_RF_REFINED / f"{Path(fname).stem}_rf_refined.png"), refined)
    print(f"  {fname}: {forest_fraction(refined):.3f}")

print(f"Refined RF masks saved to: {RESULTS_RF_REFINED}")

FIG_DIR = RESULTS_RF_REFINED / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

for (fname, img_rgb), (_, rf_raw), (_, rf_ref) in zip(images, rf_results, rf_refined_results):
    fig, ax = plt.subplots(1, 3, figsize=(14, 4), constrained_layout=True)
    ax[0].imshow(img_rgb);  ax[0].set_title(fname);   ax[0].axis("off")
    ax[1].imshow(rf_raw, cmap="gray"); ax[1].set_title("RF (post-clean)"); ax[1].axis("off")
    ax[2].imshow(rf_ref, cmap="gray"); ax[2].set_title("RF refined"); ax[2].axis("off")
    fig.savefig(str(FIG_DIR / f"{Path(fname).stem}_rf_quickpeek.png"), dpi=200)
    display(fig)  
    plt.close(fig)

print(f"Saved quick-peek panels -> {FIG_DIR}")


# Cell 13: Overlay Utility

In [ ]:
def overlay_mask(img_rgb, mask_u8, alpha=0.45, color=(60, 200, 60), resize_to="image"):
    mask = (mask_u8 > 0).astype(np.uint8)
    Hi, Wi = img_rgb.shape[:2]
    Hm, Wm = mask.shape[:2]
    if resize_to == "image" and (Hi, Wi) != (Hm, Wm):
        mask = cv2.resize(mask, (Wi, Hi), interpolation=cv2.INTER_NEAREST)
    elif resize_to == "mask" and (Hi, Wi) != (Hm, Wm):
        img_rgb = cv2.resize(img_rgb, (Wm, Hm), interpolation=cv2.INTER_AREA)
    ov = img_rgb.copy().astype(np.float32)
    ov[mask.astype(bool)] = ov[mask.astype(bool)] * (1.0 - alpha) + np.array(color, dtype=np.float32) * alpha
    return ov.astype(np.uint8)

rgb_map = {f: img for f, img in images}
rf_ref_map = {f: m for f, m in rf_refined_results}

for fname, rf_ref in rf_ref_map.items():
    img_rgb = rgb_map[fname]
    blend = overlay_mask(img_rgb, rf_ref, alpha=0.45, color=(60, 200, 60), resize_to="image")
    plt.figure(figsize=(8,8))
    plt.imshow(blend); plt.title(f"{fname} • RF refined overlay")
    plt.axis("off"); plt.show(); plt.close()


# Cell 14: Supress Scanner

In [ ]:
def paper_background_mask(gray_eq, high_pct=97, min_area=1000, dilate_k=9):
    g = ensure_uint8(gray_eq)
    thr = np.clip(int(np.percentile(g, high_pct)), 235, 252)
    white = (g >= thr).astype(np.uint8)
    white = cv2.morphologyEx(white, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    num, lab, stats, _ = cv2.connectedComponentsWithStats(white, connectivity=8)
    H, W = g.shape
    paper = np.zeros_like(white)
    for i in range(1, num):
        x, y, w_i, h_i, area = stats[i]
        touches_border = (x == 0) or (y == 0) or (x + w_i >= W-1) or (y + h_i >= H-1)
        if touches_border and area >= min_area:
            paper[lab == i] = 1

    if dilate_k:
        paper = cv2.dilate(paper, np.ones((dilate_k, dilate_k), np.uint8), iterations=1)

    content = ((1 - paper) * 255).astype(np.uint8)
    return content

rf_borderfixed_results = []
for fname, rf_mask in rf_refined_results:
    gray_eq = get_gray_eq(fname)
    content = paper_background_mask(gray_eq)
    rf_fixed = cv2.bitwise_and(rf_mask, content)
    rf_borderfixed_results.append((fname, rf_fixed))

for fname, rf_fixed in rf_borderfixed_results:
    blend = overlay_mask(rgb_map[fname], rf_fixed, alpha=0.45, color=(60,200,60), resize_to="image")
    plt.figure(figsize=(8,8))
    plt.imshow(blend); plt.title(f"{fname} • RF overlay (border suppressed)")
    plt.axis("off"); plt.show(); plt.close()


# Cell 15: Final Comparisons vs Baselines

In [ ]:
def forest_fraction(mask): 
    return (mask > 0).sum() / mask.size

def iou(a, b):
    A, B = (a > 0), (b > 0)
    inter = np.logical_and(A, B).sum()
    union = np.logical_or(A, B).sum()
    return inter / union if union else 0.0

otsu_map_final   = {f: m for f, m in otsu_results}
edges_map_final  = {f: m for f, m in edges_results}
km_map_final     = {f: m for f, m in kmeans_results}
rf_ref_map_final = {f: m for f, m in rf_refined_results}
rf_raw_map_final = {f: m for f, m in rf_results}

for fname, img_rgb in images:
    if fname not in rf_ref_map_final:
        print(f"[skip] {fname}: no RF refined mask found.")
        continue

    otsu  = otsu_map_final.get(fname)
    edges = edges_map_final.get(fname)
    km    = km_map_final.get(fname)
    rfref = rf_ref_map_final[fname]

    print(f"\n{fname}")
    print("  Fractions  | Otsu {:.3f} | Canny {:.3f} | KM {:.3f} | RF* {:.3f}"
          .format(forest_fraction(otsu), forest_fraction(edges), forest_fraction(km), forest_fraction(rfref)))
    print("  IoU vs RF* | Otsu {:.3f} | Canny {:.3f} | KM {:.3f}"
          .format(iou(rfref, otsu), iou(rfref, edges), iou(rfref, km)))
    if fname in rf_raw_map_final:
        print("  RF(raw) vs RF* IoU: {:.3f}".format(iou(rf_raw_map_final[fname], rfref)))

    fig, ax = plt.subplots(1, 5, figsize=(15, 4))
    ax[0].imshow(img_rgb); ax[0].set_title("Original");    ax[0].axis("off")
    ax[1].imshow(otsu,  cmap="gray"); ax[1].set_title("Otsu");       ax[1].axis("off")
    ax[2].imshow(edges, cmap="gray"); ax[2].set_title("Canny");      ax[2].axis("off")
    ax[3].imshow(km,    cmap="gray"); ax[3].set_title("KMeans");     ax[3].axis("off")
    ax[4].imshow(rfref, cmap="gray"); ax[4].set_title("RF refined"); ax[4].axis("off")
    plt.suptitle(fname + "   (RF* = RF refined)")
    plt.show(); plt.close()


# Cell 16: Active Learning

In [ ]:
if not ENABLE_ACTIVE:
    print("[info] Active learning is disabled (ENABLE_ACTIVE=0). Skipping Cells 17–19.")
    
if ENABLE_ACTIVE:
    UNCERTAIN_PER_IMAGE = 50
    UNCERTAIN_MIN_DIST  = 12

    def sample_uncertain_points_for_image(fname, img_rgb, clf, k=UNCERTAIN_PER_IMAGE, min_dist=UNCERTAIN_MIN_DIST):
        if 'FEATURES' in globals() and fname in FEATURES:
            F, H, W, C = FEATURES[fname]
        else:
            F = feature_stack(img_rgb, get_gray_eq(fname)); H, W, C = F.shape
        X_all = F.reshape(-1, C).astype(np.float32)
        if not hasattr(clf, "predict_proba"):
            return []
        p = clf.predict_proba(X_all)[:,1].reshape(H, W)
        u = np.abs(p - 0.5)
        ys, xs = np.unravel_index(np.argsort(u, axis=None), u.shape)

        chosen = []
        for y0, x0 in zip(ys, xs):
            if len(chosen) >= k: break
            if all((xc - x0)**2 + (yc - y0)**2 >= (min_dist**2) for (xc, yc) in chosen):
                chosen.append((x0, y0))
        return [{"fname": fname, "x": int(x), "y": int(y), "label": None, "source": "active_suggested"} 
                for (x,y) in chosen]

    ACTIVE_PROPOSALS = []
    for fname, img_rgb in images:
        ACTIVE_PROPOSALS += sample_uncertain_points_for_image(fname, img_rgb, clf,
                                k=UNCERTAIN_PER_IMAGE, min_dist=UNCERTAIN_MIN_DIST)

    print(f"Proposed {len(ACTIVE_PROPOSALS)} uncertain points across {len(images)} images.")



# Cell 17: Active Learning UI

In [ ]:
if ENABLE_ACTIVE:
    import ipywidgets as widgets
    from IPython.display import display

    PATCH_VIEW = 48  

    def get_patch(gray, x, y, half=PATCH_VIEW//2):
        H, W = gray.shape
        x0, y0 = max(0, x-half), max(0, y-half)
        x1, y1 = min(W, x+half), min(H, y+half)
        patch = gray[y0:y1, x0:x1]
        out = np.zeros((2*half, 2*half), dtype=gray.dtype)
        out[:patch.shape[0], :patch.shape[1]] = patch
        return out

    def label_active_proposals(proposals, max_to_label=100):
        global CLICK_SAMPLES
        count = 0
        status = widgets.Label()
        btn_f = widgets.Button(description="Forest", button_style="success")
        btn_n = widgets.Button(description="Non-forest", button_style="danger")
        btn_s = widgets.Button(description="Skip", button_style="")
        display(widgets.HBox([btn_f, btn_n, btn_s]), status)

        i = 0
        fig, ax = plt.subplots(figsize=(3.3,3.3))
        plt.tight_layout()

        def show_current():
            nonlocal i
            if i >= len(proposals) or count >= max_to_label:
                status.value = f"Done. Labeled {count} points."
                return
            pr = proposals[i]
            g = get_gray_eq(pr["fname"])
            patch = get_patch(g, pr["x"], pr["y"], half=PATCH_VIEW//2)
            ax.clear()
            ax.imshow(patch, cmap="gray"); ax.axis("off")
            ax.set_title(f'{pr["fname"]} @ ({pr["x"]},{pr["y"]})')
            fig.canvas.draw_idle()
            status.value = f"{i+1}/{min(len(proposals), max_to_label)}"

        def _commit(lbl):
            nonlocal i, count
            pr = proposals[i]
            if lbl is not None:
                CLICK_SAMPLES.append({"fname": pr["fname"], "x": pr["x"], "y": pr["y"],
                                      "label": int(lbl), "source": "active"})
            i += 1
            if lbl is not None: count += 1
            show_current()

        btn_f.on_click(lambda _: _commit(1))
        btn_n.on_click(lambda _: _commit(0))
        btn_s.on_click(lambda _: _commit(None))

        show_current()

    label_active_proposals(ACTIVE_PROPOSALS, max_to_label=200)


# Cell 18: Save, Retrain, Reapply

In [ ]:
if ENABLE_ACTIVE:
    CLICK_SAMPLES = dedupe_clicks(CLICK_SAMPLES)
    save_clicks(CLICK_SAMPLES)

    X, y, groups, weights = build_training_xy_weighted(CLICK_SAMPLES)
    clf, best_thr = train_rf_grouped(X, y, groups, weights)

    RESULTS_RF = os.path.join(RESULTS_ROOT, "random_forest_hybrid")
    os.makedirs(RESULTS_RF, exist_ok=True)
    rf_results = []

    for fname, img_rgb in images:
        if 'FEATURES' in globals() and fname in FEATURES:
            F, H, W, C = FEATURES[fname]
        else:
            F = feature_stack(img_rgb, get_gray_eq(fname)); H, W, C = F.shape

        X_all = F.reshape(-1, C).astype(np.float32)
        if hasattr(clf, "predict_proba"):
            pred01 = (clf.predict_proba(X_all)[:,1] >= best_thr).astype(np.uint8)
        else:
            pred01 = clf.predict(X_all).astype(np.uint8)

        pred = (pred01.reshape(H, W) * 255).astype(np.uint8)
        pred = clean_mask(pred)
        rf_results.append((fname, pred))
        cv2.imwrite(os.path.join(RESULTS_RF, f"{os.path.splitext(fname)[0]}_rf_mask.png"), pred)

    rf_refined_results = []
    for fname, pred in rf_results:
        refined = refine_mask(pred, border_pad=12, min_area=800, close_len=15, open_len=3, hole_max=2000)
        rf_refined_results.append((fname, refined))
        cv2.imwrite(os.path.join(RESULTS_RF_REFINED, f"{os.path.splitext(fname)[0]}_rf_refined.png"), refined)

    print("[retrained] Hybrid RF masks updated and refined.")
